<a href="https://colab.research.google.com/github/septgyurii/Parking_AI/blob/main/AI_PARK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tarfile
import os
from google.colab import drive

# 1) Google Drive 마운트
drive.mount('/content/drive')

# 2) 압축 파일(.tar) 경로
tar_path = "/content/drive/MyDrive/Colab Notebooks/car_yolo.tar"

# 3) 압축 해제할 경로 (tar 내부 구조대로 풀립니다)
extract_path = "/content/drive/MyDrive/Colab Notebooks"

# 4) .tar 압축 해제
with tarfile.open(tar_path, 'r') as tar:
    tar.extractall(path=extract_path)

print("✅ 압축 해제 완료:", extract_path)


Mounted at /content/drive
✅ 압축 해제 완료: /content/drive/MyDrive/Colab Notebooks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt


/content
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [ ]:
import tarfile

TAR_PATH = "/content/drive/MyDrive/Colab Notebooks/car_yolo.tar"
YOLO_DIR = "/content/yolov5"  # 공식 레포가 clone된 디렉토리

with tarfile.open(TAR_PATH, 'r') as tar:
    tar.extractall(path=YOLO_DIR)
print("✅ 커스텀 스크립트가 YOLOv5 폴더에 덮어씌워졌습니다.")


✅ 커스텀 스크립트가 YOLOv5 폴더에 덮어씌워졌습니다.


In [ ]:
!ls -R /content/yolov5 | head -n 50


/content/yolov5:
benchmarks.py
CITATION.cff
classify
CONTRIBUTING.md
data
detect.py
ds
export.py
hubconf.py
LICENSE
LICENSE.md
meta.json
models
pyproject.toml
README.md
README.zh-CN.md
requirements.txt
segment
train.py
tutorial.ipynb
utils
val.py

/content/yolov5/classify:
predict.py
train.py
tutorial.ipynb
val.py

/content/yolov5/data:
Argoverse.yaml
coco128-seg.yaml
coco128.yaml
coco.yaml
GlobalWheat2020.yaml
hyps
ImageNet1000.yaml
ImageNet100.yaml
ImageNet10.yaml
ImageNet.yaml
images
Objects365.yaml
scripts
SKU-110K.yaml
VisDrone.yaml
VOC.yaml
xView.yaml

/content/yolov5/data/hyps:


In [1]:
# 1. 환경 셋업

# 1) YOLOv5 클론 및 의존성 설치
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# 2) OCR 라이브러리 설치
!pip install easyocr

Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 18.74 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00

In [2]:
# 2. 데이터셋 준비

# convert_json_to_yolo.py
import os, json

def convert_bbox(size, box):
    # JSON이 [x_min,y_min,width,height]일 때 center 및 상대좌표 계산
    dw, dh = 1/size[0], 1/size[1]
    x, y, w, h = box
    cx = x + w/2
    cy = y + h/2
    return cx*dw, cy*dh, w*dw, h*dh

for split in ("train","val"):
    img_dir = f"/content/dataset/images/{split}"
    ann_dir = f"/content/dataset/annotations_json/{split}"
    label_dir = f"/content/dataset/labels/{split}"
    os.makedirs(label_dir, exist_ok=True)

    for ann_file in os.listdir(ann_dir):
        data = json.load(open(os.path.join(ann_dir, ann_file)))
        img_path = os.path.join(img_dir, data["filename"])
        from PIL import Image
        w,h = Image.open(img_path).size

        lines = []
        for obj in data["objects"]:
            if obj["label"] != "plate": continue
            cx,cy,nw,nh = convert_bbox((w,h), obj["bbox"])
            # plate 클래스 id=0
            lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")
        txt_path = os.path.join(label_dir, ann_file.replace(".json",".txt"))
        open(txt_path,"w").write("\n".join(lines))


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/annotations_json/train'

In [ ]:
# Colab에서 실행
!python convert_json_to_yolo.py

In [ ]:
# data.yaml 작성
train: /content/dataset/images/train
val:   /content/dataset/images/val

nc: 1
names: ["plate"]
